# Create XGBoost Regressor Model

Change working directory and import simfin

In [7]:
import os, sys
if 'notebooks' in os.getcwd():
    os.chdir('..')
sys.path.append("..")
from simfin import *
os.getcwd()

'D:\\Projects\\simfin\\simfin'

Import required libraries

In [8]:
from xgboost import XGBClassifier, XGBRegressor
from tpot import TPOTClassifier, TPOTRegressor
from sklearn.model_selection import GroupKFold
import pandas as pd
from deap import creator
from tpot.export_utils import generate_pipeline_code, expr_to_tree
import time
import numpy as np

Get simfin dataframe

In [9]:
# Extract and flaten simfin data set
if not os.path.isfile('tmp/extract.zip'):
    simfin = SimFin().extract().flatten()
else:
    simfin = SimFin().flatten()

simfin = simfin.target_reg(field='Revenues', lag=-1)

df = simfin.data_df

2019-04-11 16:53:20.596 | INFO     | sf:flatten:55 - Loading saved flattened data set ...
2019-04-11 16:53:20.928 | INFO     | sf:target_reg:108 - Adding regression target ...
2019-04-11 16:53:20.954 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for A...
2019-04-11 16:53:20.991 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for A...
2019-04-11 16:53:20.994 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for AA...
2019-04-11 16:53:20.996 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for AAL...
2019-04-11 16:53:21.000 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for AAME...
2019-04-11 16:53:21.004 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for AAN...
2019-04-11 16:53:21.007 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for AAOI...
2019-04-11 16:53:21.010 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for AAP...
2019-04-11 

2019-04-11 16:53:21.250 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for AGYS...
2019-04-11 16:53:21.254 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for AHC...
2019-04-11 16:53:21.256 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for AHCP...
2019-04-11 16:53:21.259 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for AHFD...
2019-04-11 16:53:21.262 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for AHL...
2019-04-11 16:53:21.266 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for AHPI...
2019-04-11 16:53:21.269 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for AHS...
2019-04-11 16:53:21.272 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for AHT...
2019-04-11 16:53:21.275 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for AIG...
2019-04-11 16:53:21.278 | INFO     | target_reg:by_ticker:8 - Adding target fi

2019-04-11 16:53:21.555 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for AON...
2019-04-11 16:53:21.560 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for AOS...
2019-04-11 16:53:21.565 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for AOSL...
2019-04-11 16:53:21.568 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for APA...
2019-04-11 16:53:21.572 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for APC...
2019-04-11 16:53:21.576 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for APD...
2019-04-11 16:53:21.580 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for APEVENT...
2019-04-11 16:53:21.584 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for APH...
2019-04-11 16:53:21.588 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for APIC...
2019-04-11 16:53:21.591 | INFO     | target_reg:by_ticker:8 - Adding target 

2019-04-11 16:53:21.836 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for AZZ...
2019-04-11 16:53:21.841 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for BA...
2019-04-11 16:53:21.843 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for BABL...
2019-04-11 16:53:21.846 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for BABY...
2019-04-11 16:53:21.850 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for BAC...
2019-04-11 16:53:21.852 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for BAH...
2019-04-11 16:53:21.855 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for BAX...
2019-04-11 16:53:21.859 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for BBBY...
2019-04-11 16:53:21.862 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for BBGI...
2019-04-11 16:53:21.866 | INFO     | target_reg:by_ticker:8 - Adding target fie

2019-04-11 16:53:22.089 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for BRSS...
2019-04-11 16:53:22.093 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for BRX...
2019-04-11 16:53:22.096 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for BSET...
2019-04-11 16:53:22.099 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for BSPM...
2019-04-11 16:53:22.102 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for BSQR...
2019-04-11 16:53:22.105 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for BSTC...
2019-04-11 16:53:22.108 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for BSTG...
2019-04-11 16:53:22.111 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for BSX...
2019-04-11 16:53:22.115 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for BTAI...
2019-04-11 16:53:22.118 | INFO     | target_reg:by_ticker:8 - Adding target

...

2019-04-11 16:53:26.448 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for SMG...
2019-04-11 16:53:26.451 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for SMID...
2019-04-11 16:53:26.453 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for SMIT...
2019-04-11 16:53:26.457 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for SMP...
2019-04-11 16:53:26.460 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for SMRT...
2019-04-11 16:53:26.464 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for SMSI...
2019-04-11 16:53:26.467 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for SMTC...
2019-04-11 16:53:26.470 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for SNA...
2019-04-11 16:53:26.473 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for SNAK...
2019-04-11 16:53:26.477 | INFO     | target_reg:by_ticker:8 - Adding target 

2019-04-11 16:53:26.719 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for SVU...
2019-04-11 16:53:26.723 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for SWK...
2019-04-11 16:53:26.726 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for SWKS...
2019-04-11 16:53:26.728 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for SWM...
2019-04-11 16:53:26.731 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for SWN...
2019-04-11 16:53:26.734 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for SWX...
2019-04-11 16:53:26.737 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for SWY...
2019-04-11 16:53:26.741 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for SXC...
2019-04-11 16:53:26.744 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for SXT...
2019-04-11 16:53:26.747 | INFO     | target_reg:by_ticker:8 - Adding target field

2019-04-11 16:53:26.978 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for TRLI...
2019-04-11 16:53:26.981 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for TRMB...
2019-04-11 16:53:26.984 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for TRN...
2019-04-11 16:53:26.987 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for TRNC...
2019-04-11 16:53:26.992 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for TRNS...
2019-04-11 16:53:26.995 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for TROV...
2019-04-11 16:53:26.998 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for TROW...
2019-04-11 16:53:27.002 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for TROX...
2019-04-11 16:53:27.005 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for TRT...
2019-04-11 16:53:27.007 | INFO     | target_reg:by_ticker:8 - Adding target

2019-04-11 16:53:27.233 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for USG...
2019-04-11 16:53:27.235 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for USMD...
2019-04-11 16:53:27.238 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for USNA...
2019-04-11 16:53:27.242 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for UTHR...
2019-04-11 16:53:27.245 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for UTL...
2019-04-11 16:53:27.249 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for UTMD...
2019-04-11 16:53:27.251 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for UTX...
2019-04-11 16:53:27.254 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for UVE...
2019-04-11 16:53:27.258 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for V...
2019-04-11 16:53:27.261 | INFO     | target_reg:by_ticker:8 - Adding target fiel

2019-04-11 16:53:27.492 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for WINH...
2019-04-11 16:53:27.495 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for WK...
2019-04-11 16:53:27.499 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for WLB...
2019-04-11 16:53:27.502 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for WLK...
2019-04-11 16:53:27.505 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for WLT...
2019-04-11 16:53:27.509 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for WLTW...
2019-04-11 16:53:27.511 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for WM...
2019-04-11 16:53:27.514 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for WMAL...
2019-04-11 16:53:27.518 | INFO     | target_reg:by_ticker:8 - Adding target field Revenues for WMAR...
2019-04-11 16:53:27.521 | INFO     | target_reg:by_ticker:8 - Adding target fiel

Create Dependent and Independent variables

In [20]:
X = df[pd.notnull(df['Target'])]
groups = X['Ticker']
y = X.filter(regex=r'Target.*').values.ravel()
X = X.filter(regex=r'^(?!Target).*$')
X = X.drop(['Date', 'Ticker'], axis=1)



In [23]:
print(groups.shape)
y.shape

(45298,)


(45298,)

TPOT config

In [17]:
tpot_config = {
    'xgboost.XGBRegressor': {
        'n_estimators': [10, 50, 100],
        'max_depth': [5, 10, 15, 20, None],
        'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.],
        'subsample': np.arange(0.05, 1.01, 0.05),
    },
}

# Train model
tpot = TPOTClassifier(generations=5, population_size=20, verbosity=3,
                     cv=GroupKFold(n_splits=5),
                     periodic_checkpoint_folder='tmp',
                     scoring='accuracy',
                     early_stop=5,
                     random_state=1,
                     memory='tmp',
                     warm_start=True,
                     config_dict=tpot_config,
                     )

Run TPOT

In [24]:
tpot.fit(X, y, groups=groups)

1 operators have been imported by TPOT.
Imputing missing values in feature set


D:\Anaconda3\envs\quant\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


Generation 1 - Current Pareto front scores:
-1	-inf	XGBRegressor(CombineDFs(input_matrix, input_matrix), XGBRegressor__learning_rate=1.0, XGBRegressor__max_depth=5, XGBRegressor__n_estimators=50, XGBRegressor__subsample=0.6000000000000001)



RuntimeError: There was an error in the TPOT optimization process. This could be because the data was not formatted properly, or because data for a regression problem was provided to the TPOTClassifier object. Please make sure you passed the data to TPOT correctly.

In [ ]:
print(tpot.score(X, y))

tpot.export('model.py')